In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os

In [2]:
# Path to your PlantVillage dataset folder (change this accordingly)
dataset_path = "PlantVillage"

# Image parameters
img_height, img_width = 128, 128
batch_size = 32

In [3]:
# Use ImageDataGenerator for data loading and augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,  # 20% data for validation
    rotation_range=20,
    zoom_range=0.15,
    horizontal_flip=True,
    vertical_flip=True,
    shear_range=0.15,
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)


Found 33027 images belonging to 16 classes.
Found 8249 images belonging to 16 classes.


In [4]:
num_classes = len(train_generator.class_indices)

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(img_height, img_width, 3)),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


c:\ProgramData\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 126, 126, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 61, 61, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 28, 28, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │     6,422,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 16)             │         4,112 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,521,040 (24.88 MB)

 Trainable params: 6,520,592 (24.87 MB)

 Non-trainable params: 448 (1.75 KB)

In [5]:
epochs = 5  # reduce epochs for faster turnaround

history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=epochs,
    steps_per_epoch=100,        # Train on 100 batches per epoch only (adjust as needed)
    validation_steps=20         # Validate on 20 batches only (adjust as needed)
)

c:\ProgramData\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
100/100 ━━━━━━━━━━━━━━━━━━━━ 146s 1s/step - accuracy: 0.3267 - loss: 5.9777 - val_accuracy: 0.4781 - val_loss: 5.3520
Epoch 2/5
100/100 ━━━━━━━━━━━━━━━━━━━━ 144s 1s/step - accuracy: 0.4571 - loss: 2.2357 - val_accuracy: 0.5016 - val_loss: 6.1574
Epoch 3/5
100/100 ━━━━━━━━━━━━━━━━━━━━ 147s 1s/step - accuracy: 0.4808 - loss: 2.1318 - val_accuracy: 0.3500 - val_loss: 4.1655
Epoch 4/5
100/100 ━━━━━━━━━━━━━━━━━━━━ 139s 1s/step - accuracy: 0.4735 - loss: 2.0502 - val_accuracy: 0.5141 - val_loss: 2.3374
Epoch 5/5
100/100 ━━━━━━━━━━━━━━━━━━━━ 84s 845ms/step - accuracy: 0.4670 - loss: 2.0257 - val_accuracy: 0.4016 - val_loss: 2.4326


In [6]:
train_dir = "PlantVillage"  # Change to your actual path if different
val_dir = "PlantVillage"      # Change to your actual path if different

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(128, 128),
    batch_size=16,
    class_mode='categorical'
)

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(128, 128),
    batch_size=16,
    class_mode='categorical'
)


Found 41276 images belonging to 16 classes.


NameError: name 'val_datagen' is not defined

In [ ]:
model.save("plant_disease_model.h5")
print("Model saved as plant_disease_model.h5")

Model saved as plant_disease_model.h5


In [ ]:
import numpy as np
import os
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Load model without compiling (since we only need it for inference)
model = load_model("plant_disease_model.h5", compile=False)

# Define expected image size (must match model input)
img_height, img_width = 128, 128  # adjust as per your model

def predict_image(image_path, model, target_size=(img_height, img_width)):
    if not os.path.exists(image_path):
        raise FileNotFoundError(f"Image file not found: {image_path}")

    img = load_img(image_path, target_size=target_size)
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    prediction = model.predict(img_array)
    class_idx = np.argmax(prediction)

    # Define your class labels here explicitly
    class_labels = {
        0: 'Healthy',
        1: 'Bacterial Spot',
        2: 'Early Blight',
        3: 'Late Blight',
        # Add all classes as needed
    }

    return class_labels.get(class_idx, "Unknown"), prediction[0][class_idx]

# Example usage:
image_path = "fde29807-8223-4fc5-a06b-7cc93101a6d1___GH_HL Leaf 174.JPG"

label, confidence = predict_image(image_path, model)
print(f"Predicted class: {label} with confidence {confidence:.2f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step
Predicted class: Early Blight with confidence 0.68


In [ ]:
treatment_dict = {
    'Healthy': 'No action needed. Keep monitoring regularly.',
    'Bacterial Spot': 'Use copper-based fungicides. Avoid overhead watering.',
    'Early Blight': 'Apply chlorothalonil or mancozeb. Remove infected leaves.',
    'Late Blight': 'Destroy affected plants. Use certified disease-free seeds.',
    # Add treatments for other diseases here
}

treatment = treatment_dict.get(label, "No treatment information available.")

print(f"Recommended Treatment: {treatment}")

Recommended Treatment: Apply chlorothalonil or mancozeb. Remove infected leaves.


In [ ]:
from googletrans import Translator

translator = Translator()
translated_label = translator.translate(label, src='en', dest='hi').text
translated_treatment = translator.translate(treatment, src='en', dest='hi').text

print(f"बीमारी: {translated_label}")
print(f"इलाज: {translated_treatment}")

बीमारी: जल्दी ब्लाइट
इलाज: क्लोरोथालोनिल या मैनकोज़ेब लागू करें।संक्रमित पत्तियों को हटा दें।


In [ ]:
from gtts import gTTS
import os

tts = gTTS(text=f"फसल में {translated_label} पाया गया है। इलाज: {translated_treatment}", lang='hi')
tts.save("output.mp3")
os.system("start output.mp3")  # Windows

0

In [ ]:
# Tkinter GUI placeholder:
import tkinter as tk
from tkinter import filedialog

def upload_image():
    path = filedialog.askopenfilename()
    label, confidence = predict_image(path, model)
    treatment = treatment_dict.get(label, "No data")
    result_label.config(text=f"Class: {label} ({confidence:.2f})\nTreatment: {treatment}")

root = tk.Tk()
root.title("Plant Disease Detector")

upload_btn = tk.Button(root, text="Upload Image", command=upload_image)
upload_btn.pack()

result_label = tk.Label(root, text="")
result_label.pack()

root.mainloop()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step


In [ ]:
import csv
with open("predictions_log.csv", "a", newline='') as file:
    writer = csv.writer(file)
    writer.writerow([image_path, label, confidence, treatment])

In [ ]:
import speech_recognition as sr

def get_voice_input_hindi():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("कृपया बोलें...")
        audio = recognizer.listen(source)

    try:
        text = recognizer.recognize_google(audio, language='hi-IN')
        print(f"आपने कहा: {text}")
        return text
    except sr.UnknownValueError:
        print("समझ में नहीं आया, कृपया फिर से बोलें")
        return ""
    except sr.RequestError:
        print("सर्वर से कनेक्शन नहीं हो सका")
        return ""

In [ ]:
# Trigger command
command = get_voice_input_hindi()

if "इलाज" in command or "बिमारी" in command:
    image_path = "feda8fd2-1d18-443e-a7d9-15bd6bf8ce66___RS_HL 0332.JPG" 
    label, confidence = predict_image(image_path, model)
    print(f"Predicted class: {label} with confidence {confidence:.2f}")
else:
    print("कोई प्रासंगिक कमांड नहीं पाया गया।")

कृपया बोलें...
समझ में नहीं आया, कृपया फिर से बोलें
कोई प्रासंगिक कमांड नहीं पाया गया।


In [ ]:
from gtts import gTTS
import os

def speak_hindi(text):
    tts = gTTS(text=text, lang='hi')
    tts.save("response.mp3")
    os.system("start response.mp3") 

# After prediction:
if "इलाज" in command or "बिमारी" in command:
    label, confidence = predict_image(image_path, model)
    result_text = f"पहचानी गई बिमारी: {label}। आत्मविश्वास स्तर: {confidence:.2f}"
    print(result_text)
    speak_hindi(result_text)


In [ ]:
# Sample treatment mapping (extend as needed)
treatment_advice = {
    "Pepper__bell___Bacterial_spot": "बैक्टीरियल स्पॉट के लिए कॉपर आधारित फफूंदनाशी का छिड़काव करें।",
    "Pepper__bell___healthy": "पौधा स्वस्थ है, कोई उपचार आवश्यक नहीं।",
    "Potato___Early_blight": "शुरुआती झुलसा के लिए मैनकोजेब या क्लोरोथालोनिल का प्रयोग करें।",
    "Potato___Late_blight": "लेट ब्लाइट के लिए मेटलैक्सिल आधारित दवा का छिड़काव करें।",
    "Tomato_Bacterial_spot": "बैक्टीरियल स्पॉट के लिए संक्रमित पत्तियों को हटा दें और कॉपर फफूंदनाशी का प्रयोग करें।",
    "Tomato_healthy": "पौधा स्वस्थ है। कोई इलाज जरूरी नहीं।",
    "Tomato_Late_blight": "लेट ब्लाइट के लिए मेटालैक्सिल और क्लोरोथालोनिल का मिश्रण इस्तेमाल करें।"
}

In [ ]:
if "इलाज" in command or "बिमारी" in command:
    image_path = "fe0b76dc-4263-4cda-8840-310ebe72a432___GH_HL Leaf 490.JPG"  # replace with actual image
    label, confidence = predict_image(image_path, model)
    
    # Get treatment
    advice = treatment_advice.get(label, "इस बीमारी के लिए कोई जानकारी नहीं है।")
    
    # Output to console
    print(f"बिमारी: {label} \nआत्मविश्वास: {confidence:.2f} \nइलाज: {advice}")
    
    # Speak in Hindi
    speak_hindi(f"पहचानी गई बिमारी {label} है। इलाज: {advice}")


In [ ]:
def voice_assistant_loop():
    print("कृपया बोलें... 'बिमारी', 'इलाज', या 'बंद करो'")

    while True:
        command = get_voice_input_hindi()

        if command is None:
            speak_hindi("कृपया फिर से बोलें।")
            continue

        # Trigger disease prediction
        if "बिमारी" in command or "इलाज" in command:
            image_path = "your_image.jpg"  # replace with actual image path
            try:
                label, confidence = predict_image(image_path, model)
                advice = treatment_advice.get(label, "इस बीमारी के लिए कोई जानकारी नहीं है।")
                response = f"पहचानी गई बिमारी {label} है। इलाज: {advice}"
                print(response)
                speak_hindi(response)
            except Exception as e:
                speak_hindi("चित्र में कोई समस्या है। कृपया दूसरा चित्र दें।")
                print("Error:", e)

        # Add future options here like "फसल कैलेंडर"
        elif "कैलेंडर" in command:
            speak_hindi("यह फीचर अभी विकसित किया जा रहा है।")

        # Exit loop
        elif "बंद" in command or "रुको" in command:
            speak_hindi("धन्यवाद! किसान सहायक बंद किया जा रहा है।")
            break

        else:
            speak_hindi("माफ कीजिए, मैं समझ नहीं पाया। कृपया दोबारा बोलें।")